In [1]:
import os
import random
import skimage.data
import skimage.transform
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf

# Allow image embeding in notebook
%matplotlib inline

In [3]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".jpg")]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.io.imread(f))
            labels.append(int(d))
    return images, labels


# Load training and testing datasets.
#ROOT_PATH = "/traffic"
train_data_dir = os.path.join( r'C:\Users\halil\Downloads\GTSRB\crop_dataset\crop_dataset')
test_data_dir = os.path.join( r'C:\Users\halil\Downloads\GTSRB\test_data\test_data')

images, labels = load_data(train_data_dir)

In [4]:
print("Unique Labels: {0}\nTotal Images: {1}".format(len(set(labels)), len(images)))

Unique Labels: 43
Total Images: 26640


In [5]:
## Feature extracing method ##

def extract_and_match_features(image_path, database_templates):
    # Görüntüyü gri tonlamaya dönüştür
    gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # SIFT özellik çıkarıcıyı oluştur
    sift = cv2.SIFT_create()
    
    # Keypoint'leri ve tanımlayıcıları bul
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)
    
    # Veritabanındaki şablonlarla eşleştirme
    matches = []
    for template in database_templates:
        template_keypoints, template_descriptors = template
        
        # BFMatcher kullanarak özellikleri eşleştir
        bf = cv2.BFMatcher()
        template_matches = bf.knnMatch(descriptors, template_descriptors, k=2)
        
        # İyi eşleşmeleri seç
        good_matches = []
        for match in template_matches:
            if len(match) == 2:  # Eğer eşleşme listesi beklenen uzunluktaysa
                m, n = match
                if m.distance < 1 * n.distance:
                    good_matches.append(m)
        
        # Eşleşme sayısını kaydet
        if len(good_matches) > 10:  # Örnek bir eşik değeri, ihtiyacınıza göre ayarlayabilirsiniz
            matches.append((len(good_matches), template))
    
    # Eğer uygun bir eşleşme yoksa
    if not matches:
        return None
    
    # En fazla eşleşme sayısına sahip olanı seç
    recognized_sign = max(matches, key=lambda x: x[0])[1]
    
    # En iyi eşleşen şablonun (trafik işaretinin) tanımlayıcıları döndürülür
    return recognized_sign


In [6]:
## DATABASE TEMPLATE OLUŞTURMA ##

# Database_templates'ı saklamak için liste
database_templates = []

# Meta klasöründeki tüm görüntüler için döngü
meta_folder_path = r'C:\Users\halil\Downloads\GTSRB\Meta'
for meta_filename in os.listdir(meta_folder_path):
    # Meta resminin yolunu oluştur
    meta_image_path = os.path.join(meta_folder_path, meta_filename)
    
    # Meta resmini yükle
    meta_image = cv2.imread(meta_image_path, cv2.IMREAD_GRAYSCALE)
    
    # SIFT özellik çıkarıcıyı oluştur
    sift = cv2.SIFT_create()
    
    # Keypoint'leri ve desenleri bul
    keypoints, descriptors = sift.detectAndCompute(meta_image, None)
    
    # Template'i ekle
    database_templates.append((keypoints, descriptors))

# Elde edilen tüm template'leri ekrana bas
print("Number of templates:", len(database_templates))

# Şimdi bu database_templates'ı kullanarak işleme devam edebilirsiniz


Number of templates: 43


In [7]:
# Tanıma sonuçlarını saklamak için sözlük
class_recognition_results = {}

# Meta klasöründe döngü
meta_folder_path = r'C:\Users\halil\Downloads\GTSRB\Meta'
total_recognized_count = 0
total_not_recognized_count = 0
total_count = 0
for meta_filename in os.listdir(meta_folder_path):
    # Meta resminin yolunu oluştur
    meta_image_path = os.path.join(meta_folder_path, meta_filename)
    
    # Meta resmini yükle
    meta_image = cv2.imread(meta_image_path)
    
    # Meta resminin sınıfını belirle (örneğin, dosya adından)
    class_name = os.path.splitext(meta_filename)[0]  # Dosya adından sınıf adını al
    
    # Sınıfın klasör yolunu oluştur (örneğin, 00000)
    class_folder_path = os.path.join(r'C:\Users\halil\Downloads\GTSRB\crop_dataset\crop_dataset', str(int(class_name)).zfill(5))
    
    # Sınıf klasöründeki her resim için döngü
    recognized_count = 0
    not_recognized_count = 0
    total_class_count = 0
    for image_filename in os.listdir(class_folder_path):
        # Resmin yolunu oluştur
        image_path = os.path.join(class_folder_path, image_filename)
        
        # Resmi tanı
        recognized_sign = extract_and_match_features(image_path, database_templates)
        
        # Tanıma sonucunu kontrol et
        if recognized_sign:
            recognized_count += 1
        else:
            not_recognized_count += 1
        
        total_class_count += 1
    
    # Sınıf için tanıma sonuçlarını sakla
    accuracy = recognized_count / total_class_count if total_class_count > 0 else 0.0
    class_recognition_results[class_name] = {
        'recognized_count': recognized_count,
        'not_recognized_count': not_recognized_count,
        'total_count': total_class_count,
        'accuracy': accuracy
    }
    
    # Toplam doğru ve yanlış tanınan resim sayılarını güncelle
    total_recognized_count += recognized_count
    total_not_recognized_count += not_recognized_count
    total_count += total_class_count

# Toplam doğruluk hesabı
overall_accuracy = total_recognized_count / total_count if total_count > 0 else 0.0

# Sonuçları yazdır
print("Overall Accuracy:", "{:.2%}".format(overall_accuracy))
print()
for class_name, results in class_recognition_results.items():
    print("Class:", class_name)
    print("Total Count:", results['total_count'])
    print("Recognized Count:", results['recognized_count'])
    print("Not Recognized Count:", results['not_recognized_count'])
    print("Accuracy:", "{:.2%}".format(results['accuracy']))
    print()


Overall Accuracy: 47.32%

Class: 0
Total Count: 150
Recognized Count: 102
Not Recognized Count: 48
Accuracy: 68.00%

Class: 1
Total Count: 1500
Recognized Count: 929
Not Recognized Count: 571
Accuracy: 61.93%

Class: 10
Total Count: 1350
Recognized Count: 225
Not Recognized Count: 1125
Accuracy: 16.67%

Class: 11
Total Count: 900
Recognized Count: 683
Not Recognized Count: 217
Accuracy: 75.89%

Class: 12
Total Count: 1410
Recognized Count: 653
Not Recognized Count: 757
Accuracy: 46.31%

Class: 13
Total Count: 1440
Recognized Count: 708
Not Recognized Count: 732
Accuracy: 49.17%

Class: 14
Total Count: 540
Recognized Count: 469
Not Recognized Count: 71
Accuracy: 86.85%

Class: 15
Total Count: 420
Recognized Count: 56
Not Recognized Count: 364
Accuracy: 13.33%

Class: 16
Total Count: 300
Recognized Count: 137
Not Recognized Count: 163
Accuracy: 45.67%

Class: 17
Total Count: 750
Recognized Count: 100
Not Recognized Count: 650
Accuracy: 13.33%

Class: 18
Total Count: 810
Recognized Count: